In [ ]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

from pipeline_helper import UmbrellaOptimizationCallback, allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms, set_joint_vector_field, show_center_joint_normal, show_joint_normal

from design_optimization_analysis import DesignOptimizationAnalysis

import umbrella_optimization
import umbrella_optimization_finite_diff
from umbrella_optimization import OptEnergyType

import numpy as np
import numpy.linalg as la

import pickle, gzip

from configuration import *

### Initialization

In [ ]:
name = 'sphere_cap_0.3'
input_path = '../../data/{}.json.gz'.format(name)

io, input_data, target_mesh, curr_um, thickness, target_height_multiplier = parse_input(input_path)
# target_height_multiplier = 1

# curr_um = pickle.load(gzip.open('../../output/lilium_top_four_parameters_optimized_rest_state_equilibrium_2022_01_19_18_33_target_height_factor_5.0.pkl.gz', 'r'))

#### Pin Rigid Motion

In [ ]:
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

rod_colors = get_color_field(curr_um, input_data)

# lview = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
# lview.update(scalarField = rod_colors)
# lview.show()

import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
view.getCameraParams()

view.getSize()

In [ ]:
from equilibrium_solve_analysis import EquilibriumSolveAnalysis
eqays = EquilibriumSolveAnalysis(curr_um)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

configure_umbrella_pre_deployment(curr_um, thickness, target_height_multiplier)

allGradientNorms(curr_um)

In [ ]:
break_input_angle_symmetry(curr_um)

view.showScalarField(rod_colors)

results = staged_deployment(curr_um, np.logspace(-3, 0, 4), eqm_callback, OPTS, fixedVars)

results.success

In [ ]:
eqays.plot()

### Initialize Design Optimization

In [ ]:
configure_umbrella_optimization(curr_um)

import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-8
opt_opts.verbose = 10
opt_opts.beta = 1e-6
opt_opts.niter = 600
opt_opts.verboseNonPosDef = False

results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = opt_opts, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

opt_opts.niter = 50

results.success

In [ ]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)

In [ ]:
optimizer.beta = 1 * 1e6
optimizer.gamma = 1
optimizer.eta = 1
optimizer.zeta = 1# 1e1
optimizer.iota = 1 * 1e9

In [ ]:
allDesignObjectives(optimizer)

In [ ]:
allDesignGradientNorms(optimizer)

In [ ]:
optimizer.objective.terms[-1].term.normalActivationThreshold = -2e-5

In [ ]:
optimizer.objective.terms[-1].term.normalWeight = 1
optimizer.objective.terms[-1].term.tangentialWeight = 1
optimizer.objective.terms[-1].term.torqueWeight = 0

### Gradient Finite Difference Error

In [ ]:
direction = np.random.uniform(0, 1e-3, optimizer.numParams())

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Full, minStepSize=1e-9, maxStepSize=1e-1)

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.UmbrellaForces, minStepSize=1e-9, maxStepSize=1e1)

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Elastic, minStepSize=1e-9, maxStepSize=1e-1)

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Target, minStepSize=1e-9, maxStepSize=1e0)

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.DeploymentForce, minStepSize=1e-9, maxStepSize=1e0)

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Stress, minStepSize=1e-9, maxStepSize=1e0)

### Hessian Finite Difference Errors

In [ ]:
umbrella_optimization_finite_diff.hessian_convergence_plot(optimizer, direction, umbrella_optimization_finite_diff.OptEnergyType.Full, minStepSize=1e-9, maxStepSize=1e1)

In [ ]:
umbrella_optimization_finite_diff.hessian_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.UmbrellaForces, minStepSize=1e-9, maxStepSize=1e1)

In [ ]:
umbrella_optimization_finite_diff.hessian_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Elastic, minStepSize=1e-9, maxStepSize=1e1)

In [ ]:
umbrella_optimization_finite_diff.hessian_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Target, maxStepSize=1e1, minStepSize=1e-9)

In [ ]:
umbrella_optimization_finite_diff.hessian_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.DeploymentForce, maxStepSize=1e1, minStepSize=1e-9)

In [ ]:
umbrella_optimization_finite_diff.hessian_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Stress, maxStepSize=1e1, minStepSize=1e-9)

## Stress Objective Debugging

In [ ]:
import importlib, fd_validation

In [ ]:
import fd_validation
from matplotlib import pyplot as plt
ST = elastic_rods.CrossSectionStressAnalysis.StressType
class StressLpNormFDWrapper():
    def __init__(self, stressType, p, obj): self.stressType, self.p, self.obj = stressType, p, obj
    def numVars(self):    return self.obj.numExtendedDoF()
    def getVars(self):    return self.obj.getExtendedDoFs()
    def setVars(self, x): self.obj.setExtendedDoFs(x)
    def energy(self):     return self.obj.surfaceStressLpNorm(self.stressType, self.p, True)
    def gradient(self):   return self.obj.gradSurfaceStressLpNorm(self.stressType, self.p, False, True)
    def name(self):       return self.stressType.name + f' L_{self.p} norm'
    
STs_to_test = [ST.VonMises, ST.ZStress, ST.MaxMag]
numSTs = len(STs_to_test)
counter = 1
plt.figure(figsize=(14,12))
direction = np.random.normal(size=curr_um.numExtendedDoF())
for st in STs_to_test:
    plt.subplot(numSTs, 2, counter)
    fd_validation.gradConvergencePlot(StressLpNormFDWrapper(st, 2, curr_um), direction)
    plt.subplot(numSTs, 2, counter + 1)
    fd_validation.gradConvergencePlot(StressLpNormFDWrapper(st, 6, curr_um), direction)
    counter += 2
plt.tight_layout()

In [ ]:
energyTerm = optimizer.objective.terms[0].term
tgtfitTerm = optimizer.objective.terms[1].term
stressTerm = optimizer.objective.terms[-1].term

In [ ]:
direction = np.random.normal(size=curr_um.numExtendedDoFPARL())

In [ ]:
x = curr_um.getExtendedDoFsPARL()

In [ ]:
import fd_validation
class ObjectiveTermFDWrapper():
    def __init__(self, term): self.term, self.obj = term, term.object()
    def numVars(self):    return self.obj.numExtendedDoFPARL()
    def getVars(self):    return self.obj.getExtendedDoFsPARL()
    def setVars(self, x): self.obj.setExtendedDoFsPARL(x);
    def energy(self):     return self.term.value()
    def gradient(self):   return self.term.computeGrad()
    def hessVec(self, v): return self.term.computeDeltaGrad(v)

In [ ]:
stressTerm.value()

In [ ]:
import benchmark
benchmark.reset()
fd_validation.gradConvergencePlot(ObjectiveTermFDWrapper(stressTerm))
benchmark.report()

In [ ]:
import benchmark
benchmark.reset()
fd_validation.hessConvergencePlot(ObjectiveTermFDWrapper(stressTerm), testHessVec=True)
benchmark.report()